# Reading the table

In [191]:
import pandas as pd
# Reading dataframe from excel file:
# Just like in a previous case, its a multiindex table, so we tell that 
# first two rows are headers, and index it the Order ID:
df = pd.read_excel('datasets/4.-Badly-Structured-Sales-Data-4.xlsx', header=[0, 1], index_col=0)

# cutting out unnecessary rows with headings and grand totals:
df = df.iloc[1:823]
df

Ship Mode First Class                        \
                            Segment    Consumer Corporate Home Office   
CA-2011-100293  2013-03-14 00:00:00         NaN       NaN         NaN   
CA-2011-100706  2013-12-16 00:00:00         NaN       NaN         NaN   
CA-2011-100895  2013-06-02 00:00:00         NaN       NaN         NaN   
CA-2011-100916  2013-10-21 00:00:00         NaN       NaN         NaN   
CA-2011-101266  2013-08-27 00:00:00         NaN       NaN         NaN   
...                             ...         ...       ...         ...   
US-2014-166233  2016-07-03 00:00:00         NaN       NaN         NaN   
US-2014-166611  2016-03-28 00:00:00         NaN       NaN         NaN   
US-2014-167920  2016-12-09 00:00:00         NaN       NaN         NaN   
US-2014-168116  2016-11-04 00:00:00         NaN       NaN         NaN   
US-2014-168690  2016-01-08 00:00:00         NaN       NaN         NaN   

               Same Day                       Second Class            \
               Consumer Corporate Home Office     Consumer Corporate   
CA-2011-100293      NaN       NaN         NaN          NaN       NaN   
CA-2011-100706      NaN       NaN         NaN       129.44       NaN   
CA-2011-100895      NaN       NaN         NaN          NaN       NaN   
CA-2011-100916      NaN       NaN         NaN          NaN       NaN   
CA-2011-101266      NaN       NaN         NaN        13.36       NaN   
...                 ...       ...         ...          ...       ...   
US-2014-166233      NaN       NaN         NaN          NaN       NaN   
US-2014-166611      NaN       NaN         NaN          NaN       NaN   
US-2014-167920      NaN       NaN         NaN      1827.51       NaN   
US-2014-168116      NaN   8167.42         NaN          NaN       NaN   
US-2014-168690      NaN       NaN         NaN          NaN       NaN   

                           Standard Class                        
               Home Office       Consumer Corporate Home Office  
CA-2011-100293         NaN            NaN       NaN      91.056  
CA-2011-100706         NaN            NaN       NaN         NaN  
CA-2011-100895         NaN        605.470       NaN         NaN  
CA-2011-100916         NaN            NaN   788.860         NaN  
CA-2011-101266         NaN            NaN       NaN         NaN  
...                    ...            ...       ...         ...  
US-2014-166233         NaN         24.000       NaN         NaN  
US-2014-166611         NaN            NaN    68.742         NaN  
US-2014-167920         NaN            NaN       NaN         NaN  
US-2014-168116         NaN            NaN       NaN         NaN  
US-2014-168690         NaN          2.808       NaN         NaN  

[822 rows x 13 columns]

Just like in a previous case, this table has one redundant dimension that produces a lot of NaNs.
The data needs to be consolidated.

The outright approach is to split the table on two parts:
one part is Order Date that is matched with 1 to 1 correspondence with Order ID (index column), 
and the other part is hierarchical index, that will be flattened into 3 columns (Ship Mode, Segment, and Sale).

After that, two parts of the table can be reunited merging by the index column.

# Splitting the table

In [192]:
# Slicing out the single-index part, 
# casting it to datetime simultaneously:

df_date = pd.to_datetime(df.iloc[:, 0])
# conventionally naming the column
df_date = df_date.to_frame(name='Order_Date')
df_date

,Order_Date
CA-2011-100293,2013-03-14
CA-2011-100706,2013-12-16
CA-2011-100895,2013-06-02
CA-2011-100916,2013-10-21
CA-2011-101266,2013-08-27
...,...
US-2014-166233,2016-07-03
US-2014-166611,2016-03-28
US-2014-167920,2016-12-09
US-2014-168116,2016-11-04


We have 822 rows of data, so after producing the columns they are supposed to have 822 rows as well.

In [193]:
df_sales = df.iloc[:, 1:13]
df_sales

First Class                       Same Day            \
                  Consumer Corporate Home Office Consumer Corporate   
CA-2011-100293         NaN       NaN         NaN      NaN       NaN   
CA-2011-100706         NaN       NaN         NaN      NaN       NaN   
CA-2011-100895         NaN       NaN         NaN      NaN       NaN   
CA-2011-100916         NaN       NaN         NaN      NaN       NaN   
CA-2011-101266         NaN       NaN         NaN      NaN       NaN   
...                    ...       ...         ...      ...       ...   
US-2014-166233         NaN       NaN         NaN      NaN       NaN   
US-2014-166611         NaN       NaN         NaN      NaN       NaN   
US-2014-167920         NaN       NaN         NaN      NaN       NaN   
US-2014-168116         NaN       NaN         NaN      NaN   8167.42   
US-2014-168690         NaN       NaN         NaN      NaN       NaN   

                           Second Class                       Standard Class  \
               Home Office     Consumer Corporate Home Office       Consumer   
CA-2011-100293         NaN          NaN       NaN         NaN            NaN   
CA-2011-100706         NaN       129.44       NaN         NaN            NaN   
CA-2011-100895         NaN          NaN       NaN         NaN        605.470   
CA-2011-100916         NaN          NaN       NaN         NaN            NaN   
CA-2011-101266         NaN        13.36       NaN         NaN            NaN   
...                    ...          ...       ...         ...            ...   
US-2014-166233         NaN          NaN       NaN         NaN         24.000   
US-2014-166611         NaN          NaN       NaN         NaN            NaN   
US-2014-167920         NaN      1827.51       NaN         NaN            NaN   
US-2014-168116         NaN          NaN       NaN         NaN            NaN   
US-2014-168690         NaN          NaN       NaN         NaN          2.808   

                                      
               Corporate Home Office  
CA-2011-100293       NaN      91.056  
CA-2011-100706       NaN         NaN  
CA-2011-100895       NaN         NaN  
CA-2011-100916   788.860         NaN  
CA-2011-101266       NaN         NaN  
...                  ...         ...  
US-2014-166233       NaN         NaN  
US-2014-166611    68.742         NaN  
US-2014-167920       NaN         NaN  
US-2014-168116       NaN         NaN  
US-2014-168690       NaN         NaN  

[822 rows x 12 columns]

# Converting the sparce data into more readable format

In [202]:
series = df_sales.unstack() # unstacking multiindex

# applying mask to get rid of NaN duplicates
df_filtered = series[series.isnull() == False]

# reassigning name to a newly acquired table
df_consolidated = df_filtered.to_frame().reset_index()

# renaming columns conventionally:
df_consolidated.columns.values[0] = 'Ship_Mode'
df_consolidated.columns.values[1] = 'Segment'
df_consolidated.columns.values[2] = 'Order_ID'
df_consolidated.columns.values[3] = 'Sales'

# assigning index:
df_consolidated.index = df_consolidated.Order_ID

df_consolidated


,Ship_Mode,Segment,Order_ID,Sales
Order_ID,,,,
CA-2011-103366,First Class,Consumer,CA-2011-103366,149.950
CA-2011-109043,First Class,Consumer,CA-2011-109043,243.600
CA-2011-113166,First Class,Consumer,CA-2011-113166,9.568
CA-2011-124023,First Class,Consumer,CA-2011-124023,8.960
CA-2011-130155,First Class,Consumer,CA-2011-130155,34.200
...,...,...,...,...
US-2014-129224,Standard Class,Home Office,US-2014-129224,4.608
US-2014-132031,Standard Class,Home Office,US-2014-132031,513.496
US-2014-132297,Standard Class,Home Office,US-2014-132297,598.310


# Merging tables together

Final table also has 822 rows and single index, which is a good sign.
Merging to tables on index:

In [205]:
# by default, join merges tables as a left join:
df_final = df_date.join(df_consolidated) 
df_final

,Order_Date,Ship_Mode,Segment,Order_ID,Sales
CA-2011-100293,2013-03-14,Standard Class,Home Office,CA-2011-100293,91.056
CA-2011-100706,2013-12-16,Second Class,Consumer,CA-2011-100706,129.440
CA-2011-100895,2013-06-02,Standard Class,Consumer,CA-2011-100895,605.470
CA-2011-100916,2013-10-21,Standard Class,Corporate,CA-2011-100916,788.860
CA-2011-101266,2013-08-27,Second Class,Consumer,CA-2011-101266,13.360
...,...,...,...,...,...
US-2014-166233,2016-07-03,Standard Class,Consumer,US-2014-166233,24.000
US-2014-166611,2016-03-28,Standard Class,Corporate,US-2014-166611,68.742
US-2014-167920,2016-12-09,Second Class,Consumer,US-2014-167920,1827.510
US-2014-168116,2016-11-04,Same Day,Corporate,US-2014-168116,8167.420


Again, we don't know about units of Sales column, so we will leave it with as is precision. 

# Calculating the aggregates

In [207]:
#Lastly, we calculate totals if the customer wants to have them:
print(df_final.groupby(['Ship_Mode', 'Segment']).sum())
print('Grand Total:', df_final.Sales.sum())
 

                                  Sales
Ship_Mode      Segment                 
First Class    Consumer      20802.1730
               Corporate     20792.6070
               Home Office    7737.7860
Same Day       Consumer       8132.4090
               Corporate      9907.3080
               Home Office    2977.4560
Second Class   Consumer      49724.2545
               Corporate     35243.2310
               Home Office    8791.1270
Standard Class Consumer     116922.1345
               Corporate     55942.7865
               Home Office   54748.6325
Grand Total: 391721.905
